In [1]:
import pandas as pd
import numpy as np

In [2]:
df_main = pd.read_csv('clean_data.csv')
df_imdb = pd.read_csv('clean_imdb.csv')

In [3]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [4]:
np.random.seed(42)
pdf_main = df_main.reindex(np.random.permutation(df_main.index))
pdf_imdb = df_imdb.reindex(np.random.permutation(df_imdb.index))

In [5]:
X_train, X_test, y_train, y_test = pdf_imdb['clean_text'], pdf_main['clean_text'], pdf_imdb['class'], pdf_main['class']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, Dropout, Bidirectional
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
import os
embeddings_index = {}
f = open(os.path.join('../', 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [8]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [9]:
np.random.seed(42)

In [22]:
EMBEDDING_DIM = 200
MAX_SEQUENCE_LENGTH = 100

In [10]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [23]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [24]:
pad_train = pad_sequences(seq_train, maxlen=100)
pad_test = pad_sequences(seq_test, maxlen=100)

In [25]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=False)))
# model.add(BLSTM(128, dropout=.25, recurrent_dropout=.25, return_sequences=True))
# model.add(LSTM(128, dropout=.25, recurrent_dropout=.25))
# model.add(Dense(200, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
np.random.seed(42)
model.fit(pad_train, y_train, epochs=5, batch_size=64, validation_data=(pad_test, y_test))

Train on 50000 samples, validate on 2000 samples
Epoch 1/5
50000/50000 [==============================] - 475s 9ms/step - loss: 0.4593 - acc: 0.7808 - val_loss: 0.4409 - val_acc: 0.7895
Epoch 2/5
50000/50000 [==============================] - 479s 10ms/step - loss: 0.3532 - acc: 0.8452 - val_loss: 0.4248 - val_acc: 0.8150
Epoch 3/5
50000/50000 [==============================] - 499s 10ms/step - loss: 0.3155 - acc: 0.8635 - val_loss: 0.3781 - val_acc: 0.8345
Epoch 4/5
50000/50000 [==============================] - 493s 10ms/step - loss: 0.2916 - acc: 0.8763 - val_loss: 0.3682 - val_acc: 0.8325
Epoch 5/5
50000/50000 [==============================] - 503s 10ms/step - loss: 0.2720 - acc: 0.8853 - val_loss: 0.3689 - val_acc: 0.8380


In [28]:
model.save('bidir_dense_rnn.hdf5')

In [27]:
accuracy_score(y_test, model.predict_classes(pad_test))

0.838